In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=False)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import time

Raw_Edited_v1 = "drive/My Drive/Thesis/data/Data_Engineering/Edited_Raw_Data/v1_combined_data.csv"  #combined_4x_raw_dataset_path
Raw_Edited_v2 = "drive/My Drive/Thesis/data/Data_Engineering/Edited_Raw_Data/v2_Sorted_Database.csv" #sorted_combined_dataset_path
Raw_Edited_v3 =  "drive/My Drive/Thesis/data/Data_Engineering/Edited_Raw_Data/v3_Sorted_Raw_Db_with_RUL.csv" #sorted combined with only chassis numbers present in repairs and RUL info
Raw_Edited_v4_mean = "drive/My Drive/Thesis/data/Data_Engineering/Edited_Raw_Data/v4_Sorted_Raw_Db_with_RUL_filled_mean.csv" # Filled the missing values with Mean value of respective column
Raw_Edited_v4_interpolate = "drive/My Drive/Thesis/data/Data_Engineering/Edited_Raw_Data/v4_Sorted_Raw_Db_with_RUL_filled_interpolate.csv" # Filled the missing values with linearly interpolated values
Raw_Edited_v4_norm = "drive/My Drive/Thesis/data/Data_Engineering/Edited_Raw_Data/v4_Sorted_Raw_Db_with_RUL_filled_norm.csv" # Filled the missing values by drawing from a normal distribution
Raw_Edited_v4_KNN_imputation = "drive/My Drive/Thesis/data/Data_Engineering/Edited_Raw_Data/v4_Sorted_Raw_Db_with_RUL_filled_KNN_imputation.csv" # Filled the missing values by KNN imputation

Repairs = "drive/My Drive/Thesis/data/PAC_REPAIRS_TRB_20190625_anonymized.csv" #repairs_dataset_path
Repairs_Edited_v1 = "drive/My Drive/Thesis/data/Data_Engineering/Edited_Repair_Data/v1_Sorted_Repairs.csv" #sorted_repairs_dataset_path


#### Re-arranging the columns : grouping them together for better readability
#### All Dates and RUL, Malfunction label moved to the beginning 
#### All the categorical features to the end
#### There are 69 categorical variables

In [0]:
sorted_with_rul = pd.read_csv(Raw_Edited_v3, low_memory=False)

In [0]:
# Checking how many vehicles have repair data

len(sorted_with_rul['T_CHASSIS'].unique())

445

In [0]:
# Rearranging the columns
col_names_sorted_raw_data = sorted_with_rul.columns.to_list()

RUL_Index                = col_names_sorted_raw_data.index('RUL')
Malfunction_Label_Index  = col_names_sorted_raw_data.index('Malfunction_Label')
Delivery_Date_Index = col_names_sorted_raw_data.index('DELIVERY_DATE')
Delivery_Date_1_Index = col_names_sorted_raw_data.index('DELIVERY_DATE.1')
Last_Run_Index = col_names_sorted_raw_data.index('LAST_RUN')
Last_Run_1_Index = col_names_sorted_raw_data.index('LAST_RUN.1')

cols_to_be_rearranged = ['BIWEEK_SEND_DATE', 'Malfunction_Label','RUL', 'DELIVERY_DATE', 'DELIVERY_DATE.1', 'LAST_RUN','LAST_RUN.1']
categorical_cols = ['VAR_DAX_AXLE_ARRANGEMENT','COUNTRY_OF_OPERATION', 'VAR_7MA_FRONT_AXLE_TYPE', 'VAR_DPX_ENGINE_TYPE', 'VAR_T1X_PTO_ENGINE_MOUNTED_REAR', 'VAR_4DX_FRONT_BUMPER', 'VAR_F1X_REAR_AXLE_LOAD', 'VAR_FTX_GROUND_CLEARANCE_FA', 'VAR_CKX_VEHICLE_PROFILE', 'VAR_KTX_ENGINE_EMISSION_CONTROL', 'VAR_3ZB_TRANSMISSION_PTO_TORQUE_CAP', 'VAR_QCX_TOPOGRAPHY', 'VAR_YLX_REAR_SUSPENSION_SYSTEM', 'VAR_DKX_GROSS_COMBINATION_WEIGHT', 'VAR_2RB_BATTERY_MAINTENANCE', 'VAR_FAX_STEERING_WHEEL_POS', 'VAR_2CX_CAB_VERSION', 'VAR_2DX_REAR_CAB_SUSPENSION', 'VAR_GSX_CHASSIS_HEIGHT', 'VAR_FFX_PUSHER_AXLE_FEATURES', 'VAR_5CB_AIR_SUSP_FEATURE_SOFTWARE', 'VAR_2GB_CLUTCHABLE_AIR_COMPRESSOR', 'VAR_5NX_ENGINE_BLOCK_HEATER', 'VAR_TAX_REAR_AXLE_RATIO', 'VAR_A3A_IDLE_SHUTDOWN', 'VAR_2LX_FRONT_CAB_SUSPENSION', 'VAR_RUX_GEAR_SHIFTING_SYSTEM', 'VAR_U3X_BRAKE_FRONT', 'VAR_T4X_PTO_TRANSM_MOUNTED', 'VAR_UBX_HYDRAULIC_PUMP_ENG_MTD', 'VAR_4QB_BODYBUILDER_EL_INTERFCAB_CHAS', 'VAR_REX_CRANK_CASE_VENTILATION', 'VAR_78X_TRANSPORT_CYCLE', 'VAR_T2X_TRANSMISSIONRETARDER_COOLER', 'VAR_U4X_BRAKE_DRIVE_REAR', 'VAR_FVX_FRONT_AXLE_LOAD', 'VAR_VWX_COMPRESSOR_CAPACITY', 'VAR_H1B_TURBO_APPLICATION', 'VAR_FDX_REAR_AXLE_ARRANGEMENT', 'VAR_7PX_HYDRAULIC_PUMP_GEARBOX_MOUNTED', 'VAR_7NA_FRONT_SUSPENSION_INSTALLATION', 'VAR_LKX_DRIVER_STATE_SENSING_SYSTEM', 'VAR_QWX_TURBO', 'VAR_FZX_FRONT_AXLE_ARRANGEMENT', 'VAR_V6X_RETARDER', 'VAR_K4C_VEHICLE_UPTIME_SETTINGS', 'VAR_0RA_REGENERATION_CONTROLS_DPF', 'VAR_YKX_FRONT_SUSPENSION_SYSTEM', 'VAR_T3X_PTO_ENGINE_REAR_MOUNTED', 'VAR_PJX_ALTERNATOR_CAPACITY', 'VAR_41X_PARKING_HEATER', 'VAR_D7C_WHOLE_VEHICLE_TYPE_APPROVAL', 'VAR_5JB_TELEMATICS_COMMUNICAT_GATEWAYS', 'VAR_RWX_TRANSMISSION', 'VAR_DHX_ROAD_CONDITION', 'VAR_2EX_CAB_TILT_PUMP', 'VAR_D9C_EMISSION_LEVEL', 'VAR_DDX_CHASSIS_TYPE', 'VAR_O8A_ENGINE_PTO_TORQUE_CAPACITY', 'VAR_33A_CAB_HEIGHT', 'VAR_J2A_ACTIVE_STEERING', 'VAR_N9C_ADR_CLASSIFICATION', 'VFE_0001_HORSE_POWER', 'VFE_0002_GROSS_COMBINATION_WEIGHT_BIN', 'VFE_0003_ENGINE_GENERATION', 'VFE_0004_REAR_AXLE_LOAD_BIN', 'VFE_0005_REAR_AXLE_RATIO_BIN', 'VFE_0009_LTVF_TYPE', 'VFE_0007_CAB_HEIGHT']

cols = ([col for col in cols_to_be_rearranged if col in sorted_with_rul] 
         + [col for col in sorted_with_rul if ((col not in cols_to_be_rearranged) and (col not in categorical_cols) )  ]
        + [col for col in categorical_cols])
       
sorted_with_rul = sorted_with_rul[cols]
print(sorted_with_rul.columns)


Index(['BIWEEK_SEND_DATE', 'Malfunction_Label', 'RUL', 'DELIVERY_DATE',
       'DELIVERY_DATE.1', 'LAST_RUN', 'LAST_RUN.1', 'T_CHASSIS', 'INT_BWSD',
       'LX_PMU_P1FH1_SLOPE_X_INDEX_10',
       ...
       'VAR_33A_CAB_HEIGHT', 'VAR_J2A_ACTIVE_STEERING',
       'VAR_N9C_ADR_CLASSIFICATION', 'VFE_0001_HORSE_POWER',
       'VFE_0002_GROSS_COMBINATION_WEIGHT_BIN', 'VFE_0003_ENGINE_GENERATION',
       'VFE_0004_REAR_AXLE_LOAD_BIN', 'VFE_0005_REAR_AXLE_RATIO_BIN',
       'VFE_0009_LTVF_TYPE', 'VFE_0007_CAB_HEIGHT'],
      dtype='object', length=449)


In [0]:
# Saving to .csv after re-arranging the columns
sorted_with_rul.to_csv(Raw_Edited_v3, index=False, sep=',')

In [0]:
# Before Imputing , checking the amount of missing data

print('Before Imputing, checking if any feature column has a high percentage of missing values, if yes, we can drop them')
print('Checking the percentage of missing data in the Numerical features')


numerical_features =  sorted_with_rul.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO'] 
print('Percentage of missing values in each numerical feature columns')
missing_values_ratio = (numerical_features.isnull().sum()/len(numerical_features)*100).to_list() 
print( missing_values_ratio  )
print('The column with the maximum missing value has a percentage of ',max(missing_values_ratio))


# Filters on Categorical data
print('\n'+'Checking the Categorical data for any missing data: ')
print('Percentage of missing values in each categorical feature columns')
categorical_features =  sorted_with_rul.loc[:,'VAR_DAX_AXLE_ARRANGEMENT':'VFE_0007_CAB_HEIGHT']
print(  (categorical_features.isnull().sum()/len(categorical_features)*100).to_list()  )


#### Strategies for filling the missing values in the dataset

1. Filling with mean values of respective columns
2. Interpolative imputation
3. Norm based imputation
4. KNN based imputation

In [7]:
### Using KNN imputation as suggested by Mahmoud & Reza

# First standardize data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(sorted_with_rul.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO'].values)
scaled_df = pd.DataFrame(x_scaled, columns = sorted_with_rul.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO'].columns)


# Impute Values
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
filled = imputer.fit_transform(scaled_df)
print(filled)

filled_df = pd.DataFrame(filled, columns = sorted_with_rul.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO'].columns)
sorted_with_rul.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO'] = filled_df.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO']

# Saving to .csv after filling the missing cells
sorted_with_rul.to_csv(Raw_Edited_v4_KNN_imputation, index=False, sep=',')


[[-1.65005221 -0.5450046  -0.78242084 ... -0.60671635 -1.22043352
   1.13943774]
 [-1.60041184 -0.53383402 -0.77245155 ... -0.60671635 -1.22043352
   1.13943774]
 [-1.54013424 -0.52647386 -0.76684271 ... -0.60671635 -1.22043352
   1.13943774]
 ...
 [ 1.91696293 -0.91754605 -1.10792071 ... -0.60671635 -1.22043352
  -0.87762584]
 [-1.65005221  1.93622738  0.94572239 ...  1.543879   -1.22043352
  -0.87762584]
 [-1.65005221 -0.79145474 -0.92521888 ... -0.60671635  0.87837976
  -0.87762584]]


In [0]:
## 1. Replacing the missing cells with the mean
sorted_with_rul.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO'] = sorted_with_rul.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO'].apply(lambda x: x.fillna(x.mean()),axis=0)

# Saving to .csv after filling the missing cells
sorted_with_rul.to_csv(Raw_Edited_v4_mean, index=False, sep=',')

In [0]:
# Installing the library for data imputations
!pip install autoimpute

Strategies for Imputation:

['default predictive', 'least squares', 'stochastic', 'binary logistic', 'multinomial logistic', 'bayesian least squares', 'bayesian binary logistic', 'pmm', 'lrd', 'default univariate', 'default time', 'mean', 'median', 'mode', 'random', 'norm', 'categorical', 'interpolate', 'locf', 'nocb'].

In [0]:
from autoimpute.imputations import SingleImputer, MultipleImputer


# For time series based interpolation : Doesnt work in our case
# si = SingleImputer(strategy='interpolate', 
#                    imp_kwgs={"interpolate": {"fill_strategy": "time"}},
#                    ) # imputation methods, passing through the data once



# mi = MultipleImputer() # imputation methods, passing through the data multiple times


### Strategies that work without any errors:

1. interpolate : Impute missing values using interpolation techniques.
2. norm : Impute missing data with draws from normal distribution.
3. locf : Impute missing values by carrying the last observation forward.
4. nocb : Impute missing data by carrying the next observation backward.

### Notes

*   Even though locf and nocb work, they are not suitable for our dataset, since our data is timeseries based. And we would have to make a custom locf or nocb for each Chassis number.

*   Interpolate with linear and norm works fine. Takes 1.47 seconds
*   Norm works fine and takes 1.58 seconds

*   Tried to perform interpolate using the Biweek_Send_Date as the datetime index, but since there exists some duplicates with respect to the time, that is two different chassis have same time, hence the imputation fails with a ValueError: cannot reindex from a duplicate axis


### Following strategies result in a ValueError: Mass matrix contains zeros on the diagonal.

1. PMM : Impute missing values using predictive mean matching.
2. LRD : Impute missing values using local residual draws.
3. default predictive 
4. bayesian least squares : Impute missing values using bayesian least squares regression.

### Following strategies result in a ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

4. Least squares : Impute missing values using predictions from least squares regression.
5. stochastic : Impute missing values adding error to least squares regression preds.



In [0]:
# Failed Trail with Time series based interpolation
# sorted_with_rul['BIWEEK_SEND_DATE'] = sorted_with_rul['BIWEEK_SEND_DATE'].astype('datetime64[ns]') 
# sorted_with_rul = sorted_with_rul.set_index('BIWEEK_SEND_DATE')

In [0]:
# 3. Interpolative Imputation

sorted_with_rul = pd.read_csv(Raw_Edited_v3, low_memory=False)

si = SingleImputer(strategy='interpolate') # imputation methods, passing through the data once
data_to_be_imputed = sorted_with_rul.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO']

# fit transform returns a generator by default, calculating each imputation method lazily
imputed_data = si.fit_transform(data_to_be_imputed)

# Storing all the imputed values back to the original dataframe
sorted_with_rul.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO'] = imputed_data.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO']

# Saving to .csv after filling the missing cells with linear interpolative imputation
sorted_with_rul.to_csv(Raw_Edited_v4_interpolate, index=False, sep=',')

In [0]:
# 3. Norm Imputation

sorted_with_rul = pd.read_csv(Raw_Edited_v3, low_memory=False)

si = SingleImputer(strategy='norm') # imputation methods, passing through the data once
data_to_be_imputed = sorted_with_rul.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO']

# fit transform returns a generator by default, calculating each imputation method lazily
imputed_data = si.fit_transform(data_to_be_imputed)


# Storing all the imputed values back to the original dataframe
sorted_with_rul.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO'] = imputed_data.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO']

# Saving to .csv after filling the missing cells with norm imputation
sorted_with_rul.to_csv(Raw_Edited_v4_norm, index=False, sep=',')

In [0]:
# Check if there are any nan values
sorted_with_rul[sorted_with_rul.isnull().any(axis=1)]


,BIWEEK_SEND_DATE,Malfunction_Label,RUL,DELIVERY_DATE,DELIVERY_DATE.1,LAST_RUN,LAST_RUN.1,T_CHASSIS,INT_BWSD,LX_PMU_P1FH1_SLOPE_X_INDEX_10,LX_PMU_P1FH1_SLOPE_X_INDEX_11,LX_PMU_P1FH1_SLOPE_X_INDEX_12,LX_PMU_P1FH1_SLOPE_X_INDEX_13,LX_PMU_P1FH1_SLOPE_X_INDEX_14,LX_PMU_P1FH1_SLOPE_X_INDEX_15,LX_PMU_P1FH1_SLOPE_X_INDEX_16,LX_PMU_P1FH1_SLOPE_X_INDEX_17,LX_PMU_P1FH1_SLOPE_X_INDEX_18,LX_PMU_P1FH1_SLOPE_X_INDEX_19,LX_PMU_P1FH1_SLOPE_X_INDEX_2,LX_PMU_P1FH1_SLOPE_X_INDEX_20,LX_PMU_P1FH1_SLOPE_X_INDEX_21,LX_PMU_P1FH1_SLOPE_X_INDEX_22,LX_PMU_P1FH1_SLOPE_X_INDEX_23,LX_PMU_P1FH1_SLOPE_X_INDEX_24,LX_PMU_P1FH1_SLOPE_X_INDEX_25,LX_PMU_P1FH1_SLOPE_X_INDEX_26,LX_PMU_P1FH1_SLOPE_X_INDEX_27,LX_PMU_P1FH1_SLOPE_X_INDEX_28,LX_PMU_P1FH1_SLOPE_X_INDEX_29,LX_PMU_P1FH1_SLOPE_X_INDEX_3,LX_PMU_P1FH1_SLOPE_X_INDEX_30,LX_PMU_P1FH1_SLOPE_X_INDEX_31,LX_PMU_P1FH1_SLOPE_X_INDEX_4,LX_PMU_P1FH1_SLOPE_X_INDEX_5,LX_PMU_P1FH1_SLOPE_X_INDEX_6,LX_PMU_P1FH1_SLOPE_X_INDEX_7,LX_PMU_P1FH1_SLOPE_X_INDEX_8,LX_PMU_P1FH1_SLOPE_X_INDEX_9,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_1,...,VAR_UBX_HYDRAULIC_PUMP_ENG_MTD,VAR_4QB_BODYBUILDER_EL_INTERFCAB_CHAS,VAR_REX_CRANK_CASE_VENTILATION,VAR_78X_TRANSPORT_CYCLE,VAR_T2X_TRANSMISSIONRETARDER_COOLER,VAR_U4X_BRAKE_DRIVE_REAR,VAR_FVX_FRONT_AXLE_LOAD,VAR_VWX_COMPRESSOR_CAPACITY,VAR_H1B_TURBO_APPLICATION,VAR_FDX_REAR_AXLE_ARRANGEMENT,VAR_7PX_HYDRAULIC_PUMP_GEARBOX_MOUNTED,VAR_7NA_FRONT_SUSPENSION_INSTALLATION,VAR_LKX_DRIVER_STATE_SENSING_SYSTEM,VAR_QWX_TURBO,VAR_FZX_FRONT_AXLE_ARRANGEMENT,VAR_V6X_RETARDER,VAR_K4C_VEHICLE_UPTIME_SETTINGS,VAR_0RA_REGENERATION_CONTROLS_DPF,VAR_YKX_FRONT_SUSPENSION_SYSTEM,VAR_T3X_PTO_ENGINE_REAR_MOUNTED,VAR_PJX_ALTERNATOR_CAPACITY,VAR_41X_PARKING_HEATER,VAR_D7C_WHOLE_VEHICLE_TYPE_APPROVAL,VAR_5JB_TELEMATICS_COMMUNICAT_GATEWAYS,VAR_RWX_TRANSMISSION,VAR_DHX_ROAD_CONDITION,VAR_2EX_CAB_TILT_PUMP,VAR_D9C_EMISSION_LEVEL,VAR_DDX_CHASSIS_TYPE,VAR_O8A_ENGINE_PTO_TORQUE_CAPACITY,VAR_33A_CAB_HEIGHT,VAR_J2A_ACTIVE_STEERING,VAR_N9C_ADR_CLASSIFICATION,VFE_0001_HORSE_POWER,VFE_0002_GROSS_COMBINATION_WEIGHT_BIN,VFE_0003_ENGINE_GENERATION,VFE_0004_REAR_AXLE_LOAD_BIN,VFE_0005_REAR_AXLE_RATIO_BIN,VFE_0009_LTVF_TYPE,VFE_0007_CAB_HEIGHT


In [8]:
# Filters on Numerical data
print('Filters After Imputation on Numerical data')

filled = pd.read_csv(Raw_Edited_v4_KNN_imputation, low_memory=False)
numerical_features =  filled.loc[:,'INT_BWSD':'VFE_0008_HAS_PTO'] 

# Variance Filter
variance_of_features = (numerical_features.var())
variance_threshold = 0

features_with_zero_var = variance_of_features[variance_of_features <= variance_threshold]
print(features_with_zero_var)

count_of_features_with_var_less_than_threshold= variance_of_features[variance_of_features <= variance_threshold].count()
print('Number of features with variance less than the threshold {} is'.format(variance_threshold),count_of_features_with_var_less_than_threshold)



Filters After Imputation on Numerical data
X_PST_P1C3B_128_VDLRSL2NDTHRESHOLDACTIVEDISTAN    0.0
X_PST_P1C2J_527_TLDRIVINGMONSYSTEMRECOMMENDEDG    0.0
X_PSC_P1JRU_AUTOMATIC_SOOT_DISTANCE_REGENE        0.0
X_PSC_P1JR5_AUTOMATIC_SULPHUR_DPF_REGENERA        0.0
X_PSC_P1JR2_AUTOMATIC_HC_DPF_REGENERATION_        0.0
X_PSC_P1IPW_TOTAL_USED_UREA__G                    0.0
X_PSC_P1ATW_OIL_QUALITY                           0.0
X_PSC_P1ATQ_OIL__DEGRADATION_HOURS_START          0.0
X_PSC_P1ATP_OIL__DEGRADATION_DISTANCE_STAR        0.0
X_PSC_P1AL0_DIFFERENTIATED_RSL_ENABLE             0.0
X_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_20      0.0
dtype: float64
Number of features with variance less than the threshold 0 is 11


In [0]:
# Removing the 11 features with 0 variance

for feature in (variance_of_features[variance_of_features <= variance_threshold].index):
  filled = filled.drop(str(feature), axis=1)
 


In [0]:
# Saving to csv after the Variance filter

filled.to_csv(Raw_Edited_v4_KNN_imputation, index=False, sep=',')